In [1]:
%load_ext autoreload
%autoreload 2

import pylupnt as pnt
import numpy as np
import matplotlib.pyplot as plt

output_dir = pnt.BASEDIR / "output" / "2025_FeatureMatching"

[00.03][PyLuPNT] Initializing


In [ ]:
ds_config = {
    "inherit_from": "datasets/unreal.yaml",
    "basedir": "/home/shared_ws6/data/unreal_engine/spiral_rover_lander",
    "agent": "free_agent_0",
    "cameras": ["front", "effects"],
}
# ds_config = {
#     "inherit_from": "datasets/unreal.yaml",
#     "basedir": "/home/shared_ws6/data/unreal_engine/local_traverse_v2/base_1",
#     "agent": "rover0",
#     "cameras": ["front_left"],
# }
ds = pnt.Dataset.from_config(ds_config)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

cameras = ["front", "effects"]
cameras = ["front_left"]
samples = [1, 2]  # First two samples

n = len(samples) * len(cameras)
fig, axs = plt.subplots(n, 3, figsize=(3 * 3, 3 * n))

# Ensure axs is always 2D array with shape (num_rows, num_cols)
if axs.ndim == 1:
    axs = axs[np.newaxis, :]  # Make it shape (1, N)

for i, idx in enumerate(samples):
    for j, cam in enumerate(cameras):
        row = len(cameras) * i + j
        rgb = ds[idx]["cameras"][cam]["rgb"]
        depth = ds[idx]["cameras"][cam]["depth"]
        label = ds[idx]["cameras"][cam]["label"]

        axs[row, 0].imshow(rgb)
        axs[row, 1].imshow(np.log10(depth), cmap="viridis")
        axs[row, 2].imshow(label, cmap="tab20")

for ax in axs.flatten():
    ax.axis("off")
plt.tight_layout()
plt.show()

In [ ]:
with pnt.UnrealEngine() as ue:
    ue.set_sun(azimuth=-180 * pnt.RAD, elevation=45.0 * pnt.RAD)

In [ ]:
pnt.Logger.set_level(pnt.Logger.INFO)
with pnt.UnrealEngine() as ue:
    res = ue.batch_render(
        [{"agent_id": "rover_0", "camera": "front_left", "render_type": "label"}]
    )
    print(res[0].image)

    label = ue.render("rover_0", "front_left", "label")
    print(label)

In [ ]:
import cv2

img_path = "/home/shared_ws6/data/unreal_engine/local_traverse_v2/base_1/rover0/cam_front_left/label/0.png"
img_path = (
    "/home/shared_ws6/local_traverse_v2/base_1/rover_0/cam_front_left/rgb/000000.png"
)
import os

# os.path.exists(img_path)
img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
plt.imshow(img)
img

In [ ]:
extractor = pnt.FeatureExtractor.from_config({"class": "SuperPoint"})
matcher = pnt.FeatureMatcher.from_config({"class": "SuperGlue"})

img_data1 = ds[1]["cameras"]["front"]
img_data2 = ds[2]["cameras"]["front"]
feats1 = extractor.extract(img_data1.rgb)
feats1 = pnt.filter_features_by_depth(feats1, img_data1, max_depth=1e3)
feats2 = extractor.extract(img_data2.rgb)
feats2 = pnt.filter_features_by_depth(feats2, img_data2, max_depth=1e3)
matches = matcher.match(feats1, feats2)

pnt.plot_features(
    feats1,
    feats2,
    img_data1,
    img_data2,
    matches,
    output_dir=output_dir,
    filename="rover",
    max_matches=50,
)

In [ ]:
pairs = pnt.find_all_overlapping_pairs(
    ds,
    extractor,
    camera="front",
    max_depth=100,
    thresh_dist=0.025,
    min_covisible=200,
    max_frames=100,  # Process first 100 frames, or None for all frames
    n_jobs=1,  # Use all CPU cores, or set to specific number like 4
)

In [ ]:
for idx1, idx2, n_covisible in pairs[:5]:
    img_data1 = ds[idx1]["cameras"]["front"]
    img_data2 = ds[idx2]["cameras"]["front"]

    feats1 = extractor.extract(img_data1.rgb)
    feats1 = pnt.filter_features_by_depth(feats1, img_data1, max_depth=1e3)
    feats2 = extractor.extract(img_data2.rgb)
    feats2 = pnt.filter_features_by_depth(feats2, img_data2, max_depth=1e3)
    matches = matcher.match(feats1, feats2)

    pnt.plot_features(
        feats1,
        feats2,
        img_data1,
        img_data2,
        matches,
        max_matches=100,
    )
    print(f"Plotted pair idx1={idx1}, idx2={idx2}, covisible={n_covisible}")